# Tutorial 2
We show how to train DeepScalper on BTC trading.

## Step 1: Import Packages
Modify the system path and load the corresponding packages and functions 

In [ ]:
import os
import sys

import warnings
warnings.filterwarnings("ignore")

ROOT = os.path.dirname(os.path.abspath("."))
sys.path.append(ROOT)

import torch
import argparse
import os.path as osp
from mmcv import Config
from trademaster.utils import replace_cfg_vals
from trademaster.nets.builder import build_net
from trademaster.environments.builder import build_environment
from trademaster.datasets.builder import build_dataset
from trademaster.agents.builder import build_agent
from trademaster.optimizers.builder import build_optimizer
from trademaster.losses.builder import build_loss
from trademaster.trainers.builder import build_trainer
from trademaster.transition.builder import build_transition
from trademaster.utils import plot
from trademaster.utils import set_seed
set_seed(2023)

In [1]:
#read the 5m_data.csv file  
import pandas as pd

csv = pd.read_csv('/home/krow/Desktop/coding/loltrain/deepscalp/TradeMaster/data/algorithmic_trading/BTC2/5m_data.csv')
csv

,Date,last_price,rsi,oisi,mfi,adx,high_vol,longs_closing,longs_opening,shorts_closing,shorts_opening,upper_band_diff,lower_band_diff,vwap_diff,openz,highz,lowz,closez
0,2020-09-01 20:00:00,11975.750,37.674194,48.224111,39.813471,17.024754,0,1,0,0,0,0.511704,-0.511704,0.000000e+00,-1.302852,-1.087445,-1.048698,-1.324823
1,2020-09-01 20:05:00,11969.500,32.981467,48.118494,39.163195,18.939738,1,1,0,0,0,0.482926,-0.482926,0.000000e+00,-1.323782,-1.418689,-1.732195,-1.754123
2,2020-09-01 20:10:00,11960.375,27.579985,48.120966,27.893882,20.818264,0,0,1,0,0,0.479026,-0.479026,0.000000e+00,-1.752291,-1.750487,-1.741330,-1.951232
3,2020-09-01 20:15:00,11969.250,38.183868,48.093742,27.444918,20.508287,0,0,0,1,0,0.458606,-0.458606,0.000000e+00,-1.949091,-0.905076,-1.656494,-0.509405
4,2020-09-01 20:20:00,11978.375,46.807769,48.087306,28.953984,20.024736,0,0,0,1,0,0.428232,-0.428232,0.000000e+00,-0.509169,-0.789717,-0.323943,-0.685066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366129,2024-03-02 23:35:00,62065.750,62.863808,51.678115,55.545914,15.521840,0,1,0,0,0,0.430897,-0.430897,1.172298e-14,0.843616,1.208074,0.985107,1.238855
366130,2024-03-02 23:40:00,62093.800,65.448899,51.696692,56.932071,15.538116,0,0,1,0,0,0.431008,-0.431008,0.000000e+00,1.238855,1.332066,1.519130,1.420015
366131,2024-03-02 23:45:00,62081.300,63.333116,51.590870,58.380620,15.139127,0,1,0,0,0,0.432017,-0.432017,0.000000e+00,1.420015,1.044015,1.258167,1.286972
366132,2024-03-02 23:50:00,62074.900,62.223988,51.597417,58.704881,14.768636,0,0,0,0,1,0.429426,-0.429426,0.000000e+00,1.286972,0.988174,1.299976,1.082321


In [ ]:
#split into train, validate and test and save as csv in same folder
train = csv[:int(0.7*len(csv))]
validate = csv[int(0.7*len(csv)):int(0.85*len(csv))]
test = csv[int(0.85*len(csv)):]
train.to_csv('/home/krow/Desktop/coding/loltrain/deepscalp/TradeMaster/data/algorithmic_trading/BTC2/train.csv', index=False)
validate.to_csv('/home/krow/Desktop/coding/loltrain/deepscalp/TradeMaster/data/algorithmic_trading/BTC2/validate.csv', index=False)
test.to_csv('/home/krow/Desktop/coding/loltrain/deepscalp/TradeMaster/data/algorithmic_trading/BTC2/test.csv', index=False)

## Step 2: Load Configs
Load default config from the file `configs/algorithmic_trading/algorithmic_trading_BTC_deepscalper_deepscalper_adam_mse.py`

In [ ]:

parser = argparse.ArgumentParser(description='Open s1m data')
parser.add_argument("--config", default=osp.join(ROOT, "configs", "algorithmic_trading", "deepsn1per.py"),
                    help="download datasets config file path")
parser.add_argument("--task_name", type=str, default="train")
args, _= parser.parse_known_args()

cfg = Config.fromfile(args.config)
task_name = args.task_name
cfg = replace_cfg_vals(cfg)

In [ ]:
cfg

## Step 3: Build Dataset

In [ ]:
dataset = build_dataset(cfg)

## Step 4: Build Reinforcement Learning Environments
Build environments based on cfg and predefined dataset

In [ ]:
train_environment = build_environment(cfg, default_args=dict(dataset=dataset, task="train"))
valid_environment = build_environment(cfg, default_args=dict(dataset=dataset, task="valid"))
test_environment = build_environment(cfg, default_args=dict(dataset=dataset, task="test"))

In [ ]:
train_environment.df.head()

## Step 5: Build Net 
Update information about the state and action dimension. Create networks and optimizer for DeepScalper


In [ ]:
action_dim = train_environment.action_dim
state_dim = train_environment.state_dim

cfg.act.update(dict(action_dim=action_dim, state_dim=state_dim))
act = build_net(cfg.act)
act_optimizer = build_optimizer(cfg, default_args=dict(params=act.parameters()))
if cfg.cri:
    cfg.cri.update(dict(action_dim=action_dim, state_dim=state_dim))
    cri = build_net(cfg.cri)
    cri_optimizer = build_optimizer(cfg, default_args=dict(params=cri.parameters()))
else:
    cri = None
    cri_optimizer = None

## Step 6: Build Loss

In [ ]:
criterion = build_loss(cfg)

## Step 7: Build Transition

In [ ]:
transition = build_transition(cfg)


## Step 8: Build Agent

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
agent = build_agent(cfg, default_args=dict(action_dim = action_dim,state_dim = state_dim,act = act,cri = cri,act_optimizer = act_optimizer,cri_optimizer = cri_optimizer, criterion = criterion,transition = transition,device=device))

## Step 9: Build Trainer
Build trainer from config and create work directionary to save the result, model and config

In [ ]:
trainer = build_trainer(cfg, default_args=dict(train_environment=train_environment,valid_environment=valid_environment,test_environment=test_environment,agent=agent,device=device))

## Step 10: Train the RL Agent
Train the trainer based on the config and save results in workdir

In [ ]:
trainer.train_and_valid()

## Step 11: RL Agent Testing

In [ ]:
trainer.test();

In [ ]:
plot(trainer.test_environment.save_asset_memory(),alg="DeepScalper")